In [1]:
import numpy as np

In [2]:
def normalize(x):
    x = x/x.sum()
    return x

In [3]:
from sklearn import preprocessing
def one_hot_encode(x):
    lb = preprocessing.LabelBinarizer()
    lb.fit(x)
    return lb.transform(x)

In [4]:
datasets = ['vision_hackathon_drive/vision_hackathon_drive/Amy_Adams',
'vision_hackathon_drive/vision_hackathon_drive/bradley_cooper',
'vision_hackathon_drive/vision_hackathon_drive/Cobie_Smulders',
'vision_hackathon_drive/vision_hackathon_drive/gal_gadot',
'vision_hackathon_drive/vision_hackathon_drive/hrithik_roshan',
'vision_hackathon_drive/vision_hackathon_drive/hugh_jackman',
'vision_hackathon_drive/vision_hackathon_drive/Isla_Fisher',
'vision_hackathon_drive/vision_hackathon_drive/Katrina_Kaif',
'vision_hackathon_drive/vision_hackathon_drive/roger_federer',
'vision_hackathon_drive/vision_hackathon_drive/Saif_Ali_Khan',
'vision_hackathon_drive/vision_hackathon_drive/scarlett_johansson',
'vision_hackathon_drive/vision_hackathon_drive/shahid_kapoor',
'vision_hackathon_drive/vision_hackathon_drive/shraddha_kapoor',
'vision_hackathon_drive/vision_hackathon_drive/tom_hardy',
'vision_hackathon_drive/vision_hackathon_drive/vani_kapoor']

In [5]:
from glob import glob

In [6]:
from PIL import Image

In [7]:
import cv2
def get_frame(image_file):
    image = cv2.imread(image_file,cv2.IMREAD_COLOR)
    #print(image.shape)
    image = cv2.resize(image, (255,255))
    feature = np.array(image, dtype=np.float32)
    return feature

In [8]:
features = []
labels = []
for data_dir in datasets:
    paths = glob("{}/*.jpg".format(data_dir))
    label = data_dir.split('/')[-1]
    #print(label)
    for path in paths:
        features.append(get_frame(path))
        labels.append(label)

In [9]:
features = np.array(features)
labels = np.array(labels)

In [10]:
features = features/255

In [11]:
features = np.array([normalize(feature) for feature in features])

In [12]:
labels = one_hot_encode(labels)

In [13]:
print('Feature dimensions : {}'.format(features.shape))
print('Labels dimensions : {}'.format(labels.shape))

Feature dimensions : (1664, 255, 255, 3)
Labels dimensions : (1664, 15)


In [14]:
labels = labels.astype(np.float32)

In [15]:
from sklearn.model_selection import train_test_split
train_features, valid_features, train_labels, valid_labels = train_test_split(
    features,
    labels,
    test_size=0.05,
    random_state=832289)

In [16]:
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
class Model:
    def __init__(self, nh,nw,nc, ny):
        with tf.variable_scope('inputs'):
            self.X = tf.placeholder(tf.float32, [None,nh,nw, nc], name='X')
            self.Y = tf.placeholder(tf.float32,[None,ny],name='Y')
    
    def initialize(self):
        W1 = tf.Variable(tf.truncated_normal([4,4,3,8], stddev=0.1), name="W1")
        W2 = tf.Variable(tf.truncated_normal([2,2,8,16], stddev=0.1), name="W2")
        print('Shape of X : ', self.X.get_shape().as_list())
        Z1 = tf.nn.conv2d(self.X, W1, strides=[1, 1, 1, 1], padding='SAME')
        print('Shape of Z1 : ', Z1.get_shape().as_list())
        A1 = tf.nn.relu(Z1)
        print('Shape of A1 : ', A1.get_shape().as_list())
        P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], strides = [1, 8, 8, 1], padding='SAME')
        print('Shape of P1 : ', P1.get_shape().as_list())
        Z2 = tf.nn.conv2d(P1, W2, strides=[1, 1, 1, 1], padding='SAME')
        print('Shape of Z2 : ', Z2.get_shape().as_list())
        A2 = tf.nn.relu(Z2)
        print('Shape of A2 : ', A2.get_shape().as_list())
        P2 = tf.nn.max_pool(A2, ksize = [1, 4, 4, 1], strides = [1, 4, 4, 1], padding='SAME')
        print('Shape of P2 : ', P2.get_shape().as_list())
        P = tf.keras.layers.flatten(P2)
        print('Shape of P : ', P.get_shape().as_list())
        self.Z3 = tf.contrib.layers.fully_connected(P, 15, activation_fn=None)
        print('Shape of Z3 : ', self.Z3.get_shape().as_list())
                
    def add_loss(self):
        with tf.variable_scope('loss'):
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.Z3, labels=self.Y)
            self.loss = tf.reduce_mean(cross_entropy)
        
    def add_optimizer(self,global_step,learning_rate=1.e-3,adam_beta1=0.9,adam_beta2=0.999):
        with tf.variable_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate, adam_beta1, adam_beta2)
            gradients, variables = zip(*optimizer.compute_gradients(self.loss))
            self.gradients = gradients
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                self.optimize = optimizer.apply_gradients(zip(clipped_gradients, variables),global_step=global_step)
                
    def add_accuracy(self):
        # Determine if the predictions are correct
        is_correct_prediction = tf.equal(tf.argmax(self.Z3, 1), tf.argmax(self.Y, 1))
        # Calculate the accuracy of the predictions
        accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [18]:
model = Model(255,255,3,15)

In [19]:
global_step = tf.Variable(0, name="global_step", trainable=False)
model.initialize()
model.add_loss()
model.add_optimizer(global_step)
model.add_accuracy()

Instructions for updating:
Colocations handled automatically by placer.
Shape of X :  [None, 255, 255, 3]
Shape of Z1 :  [None, 255, 255, 8]
Shape of A1 :  [None, 255, 255, 8]
Shape of P1 :  [None, 32, 32, 8]
Shape of Z2 :  [None, 32, 32, 16]
Shape of A2 :  [None, 32, 32, 16]
Shape of P2 :  [None, 8, 8, 16]

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Shape of P :  [None, 1024]
Shape of Z3 :  [None, 15]
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [21]:
sess = tf.Session()

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
num_of_epochs = 100

for epoch in range(num_of_epochs):
    batches = batch_iter(list(zip(train_features,train_labels)),32,100)
    for batch in batches:
        x_batch,y_batch = zip(*batch)
        current_step = tf.train.global_step(sess, global_step)
        step,loss,_ = sess.run([global_step,model.loss,model.optimize],feed_dict = {model.X:x_batch, model.Y:y_batch})
        if (step % 100 == 0):
            print('Step : {%d} , Loss = {%f}' % (step,loss))
    if epoch % 10 == 0:
        training_accuracy = sess.run(self.accuracy,feed_dict={model.X:train_features,model.Y:train_labels})
        validation_accuracy = sess.run(self.accuracy,feed_dict={model.X:valid_labels, model.Y:valid_labels})
        print('Training accuracy at {}'.format(training_accuracy))
        print('Validation accuracy at {}'.format(validation_accuracy))

Step : {100} , Loss = {2.704194}
Step : {200} , Loss = {2.664703}
Step : {300} , Loss = {2.688957}
Step : {400} , Loss = {2.694161}
Step : {500} , Loss = {2.612667}
Step : {600} , Loss = {2.678887}
Step : {700} , Loss = {2.592733}
Step : {800} , Loss = {2.558969}
Step : {900} , Loss = {2.552173}
Step : {1000} , Loss = {2.667575}
Step : {1100} , Loss = {2.548392}
Step : {1200} , Loss = {2.583253}
Step : {1300} , Loss = {2.549084}
Step : {1400} , Loss = {2.528803}
Step : {1500} , Loss = {2.675372}
Step : {1600} , Loss = {2.577952}
Step : {1700} , Loss = {2.359047}
Step : {1800} , Loss = {2.520584}
Step : {1900} , Loss = {2.480792}
Step : {2000} , Loss = {2.486034}
Step : {2100} , Loss = {2.804366}
Step : {2200} , Loss = {2.311068}
Step : {2300} , Loss = {2.598475}
Step : {2400} , Loss = {2.592263}
Step : {2500} , Loss = {2.320297}
Step : {2600} , Loss = {2.134440}
Step : {2700} , Loss = {1.987363}



KeyboardInterrupt

